## 데이터수집

In [1]:
from sklearn.datasets import load_diabetes
data_diabetes = load_diabetes()
features, label = data_diabetes.data, data_diabetes.target
features.shape, label.shape

((442, 10), (442,))

## 데이터 전처리
- 데이터 분석

## 데이터 분할
- train, test, validation

In [2]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_label, test_label = train_test_split(features, label, test_size=0.2, random_state=42)

In [3]:
import torch

In [4]:
train_features_tensor = torch.tensor(train_features, dtype=torch.float32)
train_label_tensor = torch.tensor(train_label, dtype=torch.float32).view(-1, 1)    # 행렬 모양으로 변경
# test_features_tensor = torch.tensor(test_features)
# test_label_tensor = torch.tensor(test_label)
type(train_features_tensor), type(train_label_tensor), train_label_tensor.shape

(torch.Tensor, torch.Tensor, torch.Size([353, 1]))

## 모델 학습

In [5]:
torch.nn.Module?

In [6]:
# Neural Network model linear regression
class LinearRegressionNNM(torch.nn.Module):
    def __init__(self, input_dim):    # input_dim:features 열 갯수
        super(LinearRegressionNNM, self).__init__()
        # super(self).__init__()
        self.hidden_1 = torch.nn.Linear(input_dim, 64)
        self.hidden_2 = torch.nn.Linear(64, 32)
        self.hidden_3 = torch.nn.Linear(32, 16)
        self.output = torch.nn.Linear(16, 1)
        self.relu = torch.nn.ReLU()  # Activation function

    def forward(self, x):
        x = self.relu(self.hidden_1(x))  # hidden layer 1
        x = self.relu(self.hidden_2(x))  # hidden layer 2
        x = self.relu(self.hidden_3(x))  # hidden layer 3
        out = self.output(x)   # result
        return out


In [7]:
train_features_tensor.shape, train_label_tensor.shape

(torch.Size([353, 10]), torch.Size([353, 1]))

In [8]:
model = LinearRegressionNNM(10)

In [9]:
model

LinearRegressionNNM(
  (hidden_1): Linear(in_features=10, out_features=64, bias=True)
  (hidden_2): Linear(in_features=64, out_features=32, bias=True)
  (hidden_3): Linear(in_features=32, out_features=16, bias=True)
  (output): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
)

In [10]:
criterion = torch.nn.MSELoss()  # Loss function
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  # optimizer function
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # optimizer function

In [11]:
## 반복 학습
# for epoch in range(10):
# for epoch in range(1000):
for epoch in range(10000):
    # Forward pass: Compute predicted y by passing x to the model
    pred_y = model(train_features_tensor)

    # Compute and print loss
    loss = criterion(pred_y, train_label_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
      print('epoch: {}, loss: {}'.format(epoch, loss.item()))

epoch: 0, loss: 29777.78515625
epoch: 1000, loss: 2967.0478515625
epoch: 2000, loss: 2874.95849609375
epoch: 3000, loss: 2798.280029296875
epoch: 4000, loss: 2690.76904296875
epoch: 5000, loss: 2422.831787109375
epoch: 6000, loss: 1992.7197265625
epoch: 7000, loss: 1277.6826171875
epoch: 8000, loss: 750.95263671875
epoch: 9000, loss: 556.1344604492188


In [12]:
# 전체 weight와 bies
# model.state_dict()
model.state_dict().keys()

odict_keys(['hidden_1.weight', 'hidden_1.bias', 'hidden_2.weight', 'hidden_2.bias', 'hidden_3.weight', 'hidden_3.bias', 'output.weight', 'output.bias'])

In [13]:
model.state_dict()['hidden_1.weight'].shape, model.state_dict()['hidden_1.bias'].shape, 

(torch.Size([64, 10]), torch.Size([64]))

In [14]:
model.state_dict()['hidden_2.weight'].shape, model.state_dict()['hidden_2.bias'].shape, 

(torch.Size([32, 64]), torch.Size([32]))

In [15]:
model.state_dict()['output.weight'].shape, model.state_dict()['output.bias'].shape, 

(torch.Size([1, 16]), torch.Size([1]))

## 모델 평가

In [16]:
model.eval()

LinearRegressionNNM(
  (hidden_1): Linear(in_features=10, out_features=64, bias=True)
  (hidden_2): Linear(in_features=64, out_features=32, bias=True)
  (hidden_3): Linear(in_features=32, out_features=16, bias=True)
  (output): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
)

In [17]:
with torch.no_grad():   # 학습 목적 아닌 평가 목적 위해 고정
    pred_y = model(train_features_tensor)
    loss = criterion(pred_y, train_label_tensor)  # 예측도 같은 Loss function
    print('loss: {}'.format(loss.item()))

loss: 430.0226745605469


In [18]:
# 결과 값(예측값, 원래값)
pred_y[0], train_label[0]

(tensor([126.3260]), 144.0)

## 모델 배포

In [19]:
# 모델 weight와 bias 파일로 저장(checkpoints)
torch.save(model.state_dict(), "linear_regression_nnm.pth")